In [1]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

import numpy as np
import pandas as pd
import math
import pprint
pp = pprint.PrettyPrinter(indent=4)
from decimal import Decimal
import datetime
import pprint
import csv

In [2]:
#prepare datas
import csv
with open("funds41.csv", encoding='utf-8') as r_file:
    
    file_reader = csv.reader(r_file, delimiter = ",")
    counter=0
    datas=[]
    for row in file_reader:
      
        if counter==0:
           
            counter = 1
        else:
            datas.append([row[0], float(row[1]),float(row[2]),float(row[3]),float(row[4]),float(row[5]),float(row[6]),float(row[7]),float(row[8]) ])
            #step	pr1	pr2	pr3	pr4	TVL1	TVL2	TVL3	TVL4


pp.pprint(datas[1][2])  

FileNotFoundError: [Errno 2] No such file or directory: 'funds41.csv'

In [ ]:
#make function for computind vaults shares accoding vaults price velosity, TVL and velosity of  changing TVL of vaults
total_funds=400000
delta=3
def calc_price_velosity(datas, i, delta):#datas, step, number of dates
    arr=[]
    
    arr=[(datas[i+delta][1]-datas[i][1])/delta,
         (datas[i+delta][2]-datas[i][2])/delta,
         (datas[i+delta][3]-datas[i][3])/delta,
         (datas[i+delta][4]-datas[i][4])/delta]
    return arr    
print ('calc_price_velosity = ',calc_price_velosity(datas, 1, 3))   # for check

def calc_tvl_velosity(datas, i, delta):#datas, step, number of dates
    arr=[]
    
    arr=[(datas[i+delta][5]-datas[i][5])/delta,
         (datas[i+delta][6]-datas[i][6])/delta,
         (datas[i+delta][7]-datas[i][7])/delta,
         (datas[i+delta][8]-datas[i][8])/delta]
    return arr    
print ('calc_tvl_velosity = ', calc_tvl_velosity(datas, 1, 3))   # for check
        

def calc_shares_tvl(datas, step, total_funds):#computing shares according TVL
    totalTVL= datas[step][5]+ datas[step][6]+ datas[step][7]+ datas[step][8]
    funds = [total_funds*datas[step][5]/totalTVL, total_funds*datas[step][6]/totalTVL, 
             total_funds*datas[step][7]/totalTVL, total_funds*datas[step][8]/totalTVL]
    return funds
print ('calc_shares_tvl = ',calc_shares_tvl(datas, 1, total_funds))   # for check

def calc_shares_tvl_velosity(datas, step, total_funds, delta):#computing shares according TVL gain speed
    totalTVLvel= calc_tvl_velosity(datas, step, delta)[0]+calc_tvl_velosity(datas, step, delta)[1]+calc_tvl_velosity(datas, step, delta)[2]+calc_tvl_velosity(datas, step, delta)[3]
    funds = [total_funds*calc_tvl_velosity(datas, step, delta)[0]/totalTVLvel, total_funds*calc_tvl_velosity(datas, step, delta)[1]/totalTVLvel, 
             total_funds*calc_tvl_velosity(datas, step, delta)[2]/totalTVLvel, total_funds*calc_tvl_velosity(datas, step, delta)[3]/totalTVLvel]
    return funds
print ('calc_shares_tvl_velosity = ',calc_shares_tvl_velosity(datas, 1, total_funds, delta))   # for check

def calc_shares_price_velosity(datas, step, total_funds, delta):#computing shares according velosity of gain of vault prices
    total_velosity = calc_price_velosity(datas, step, delta)[0]+calc_price_velosity(datas, step, delta)[1]+calc_price_velosity(datas, step, delta)[2]+calc_price_velosity(datas, step, delta)[3]
    funds = [total_funds*calc_price_velosity(datas, step, delta)[0]/total_velosity, 
             total_funds*calc_price_velosity(datas, step, delta)[1]/total_velosity, 
             total_funds*calc_price_velosity(datas, step, delta)[2]/total_velosity, 
             total_funds*calc_price_velosity(datas, step, delta)[3]/total_velosity]
    return funds
print ('calc_shares_price_velosity = ', calc_shares_price_velosity(datas, 1, total_funds, delta))   # for check


In [ ]:
#prepare datas
total_funds=400000 # total funds in our operating
delta=1 #step of changing shares in Vaults 
funds1 =[]
funds2=[]
funds3=[]
funds4=[]
datas1=[]
weights=[1/3,1/3,1/3] #weights for computing influence of calculations of shares
for i in range (0 ,len(datas)-delta, 1):
    if (i % delta) ==0:
        funds1.append(calc_shares_tvl(datas, i, total_funds))
        funds2.append(calc_shares_tvl_velosity(datas, i, total_funds, delta))
        funds3.append(calc_shares_price_velosity(datas, i, total_funds, delta))
        datas1.append([calc_shares_tvl(datas, i, total_funds), calc_shares_tvl_velosity(datas, i, total_funds, delta),calc_shares_price_velosity(datas, i, total_funds, delta)])

for i in range (0 ,len(funds1)-1, 1):
    funds4.append([funds1[i][0]*weights[0]+funds2[i][0]*weights[1]+funds2[i][0]*weights[2],
                   funds1[i][1]*weights[0]+funds2[i][1]*weights[1]+funds2[i][1]*weights[2],
                   funds1[i][2]*weights[0]+funds2[i][2]*weights[1]+funds2[i][2]*weights[2],
                   funds1[i][3]*weights[0]+funds2[i][3]*weights[1]+funds2[i][3]*weights[2]])
#pp.pprint(funds4)


In [ ]:
import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
x = np.arange(0, len(funds4)-1, 1)
x1=[]
x2=[]
x3=[]
x4=[]
for i in range(0,len(funds4)-1):
    x1.append(funds4[i][0])
    x2.append(funds4[i][1])
    x3.append(funds4[i][2])
    x4.append(funds4[i][3])

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=x1, name='<sup>Amount in Vault 1</sup>'))
fig.add_trace(go.Scatter(x=x, y=x2, name='<sup>Amount in Vault 2</sup>'))
fig.add_trace(go.Scatter(x=x, y=x3, name='<sup>Amount in Vault 3</sup>'))
fig.add_trace(go.Scatter(x=x, y=x4, name='<sup>Amount in Vault 4</sup>'))
fig.update_layout(title_text='<b>Amount of Money in Four Vaults </b>')
fig.update_xaxes(title_text='Step')
fig.show()

In [ ]:
pp.pprint(datas1[1][2][3])

In [ ]:
a=1/3 #parametr a in [0, 1]
b=1/3 #parametr b in [0, 1]
weights=[1-a-b,b,a] #
total_funds=400000 # total funds in our operating

def frange(start, end, step):
    i = start
    while i < end:
        yield i
        i += step

def changeable_shares(a, b, datas ):
    datas2=[]
    for i in range (0 ,len(datas1)-1, 1):
        datas2.append([datas1[i][0][0]*(1-a-b)+datas1[i][1][0]*b+datas1[i][2][0]*a,
                   datas1[i][0][1]*(1-a-b)+datas1[i][1][1]*b+datas1[i][2][1]*a,
                   datas1[i][0][2]*(1-a-b)+datas1[i][1][2]*b+datas1[i][2][2]*a,
                   datas1[i][0][3]*(1-a-b)+datas1[i][1][3]*b+datas1[i][2][3]*a])
    return datas2
#print('changeable_shares = ',changeable_shares(a, b, datas1 )[1][3] )
#print('datas=', datas[3][4])
def calc_max_profit(total_funds, datas, datas0):
    total_profit = 0
    
    for i in range (0 ,len(datas1)-1, 1):
        profit = 0
        for a in frange (0.01 , 1, 0.1):
            for b in frange (0.01 , 1, 0.1):
                #ch=changeable_shares(a, b, datas1)[i][0]+changeable_shares(a, b, datas1)[i][1]+changeable_shares(a, b, datas1)[i][2]+changeable_shares(a, b, datas1)[i][3]
                prof1 = changeable_shares(a, b, datas1)[i][0]*(datas0[i+1][1]-datas0[i][1])/datas0[i][1]+changeable_shares(a, b, datas1)[i][1]*(datas0[i+1][2]-datas0[i][2])/datas0[i][2]+changeable_shares(a, b, datas1 )[i][2]*(datas0[i+1][3]-datas0[i][3])/datas0[i][3]+changeable_shares(a, b, datas1 )[i][3]*(datas0[i+1][4]-datas0[i][4])/datas0[i][4]
                if profit<prof1:
                    profit=prof1
                    #print('profit= ', profit)
        total_profit = total_profit+profit       
    return total_profit
print('calc_max_profit = ',calc_max_profit(total_funds, datas1, datas ) )

In [ ]:
#Строим график 3D
def arr_profit(total_funds, datas, datas0):
    arr = []
    
    for i in range (0 ,len(datas1)-1, 1):
        profit = 0
        for a in frange (0.01 , 1, 0.1):
            for b in frange (0.01 , 1, 0.1):
                #ch=changeable_shares(a, b, datas1)[i][0]+changeable_shares(a, b, datas1)[i][1]+changeable_shares(a, b, datas1)[i][2]+changeable_shares(a, b, datas1)[i][3]
                prof1 = changeable_shares(a, b, datas1)[i][0]*(datas0[i+1][1]-datas0[i][1])/datas0[i][1]+changeable_shares(a, b, datas1)[i][1]*(datas0[i+1][2]-datas0[i][2])/datas0[i][2]+changeable_shares(a, b, datas1 )[i][2]*(datas0[i+1][3]-datas0[i][3])/datas0[i][3]+changeable_shares(a, b, datas1 )[i][3]*(datas0[i+1][4]-datas0[i][4])/datas0[i][4]
                arr.append([a,b, prof1])      
    return arr
arr1=arr_profit(total_funds,datas1, datas)
#pp.pprint(arr1)

In [ ]:
%config InlineBackend.figure_format = 'svg'
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt

#create 3d axes
fig = plt.figure()
ax = plt.axes(projection='3d')

x=[]
y=[]
z=[]
for i in range(0,len(arr1)-1,1):
    x.append(arr1[i][0])
    y.append(arr1[i][1])
    z.append(arr1[i][2])

ax.plot3D(x, y, z, 'red')
#fig = plt.figure(figsize=(10,10))
#axes = fig.gca(projection='3d')
#surf = axes.plot_surface(x, y, z, cmap='coolwarm',linewidth=0, antialiased=False)
plt.show()